# GPU Usage:

**Recommendation:** Finish coding and debugging on CPU; only use the GPU in the end to get the final results.

**Important.** Things to keep in mind:

- **The GPU charges by the minute. Only use it when required and remember to delete the GPU job as soon as you're done.** If you use up the credits given to you, we will not be able to provide you with more credits.
- To check which GPUs are assigned to you and to delete the GPU job, go [here](https://console.cloud.google.com/marketplace/product/colab-marketplace-image-public/colab) and click "View Deployments". **Disconnecting the GPU from Colab does not delete the instance.** You need to delete it at the provided link.
- When selecting the GPU, select the region to be somewhere in the US.
- For GPU type, request either T4 or P100. In our experience, P100s are more easily available.
- If the GPU resource is not available, you can try to reserve a GPU in another region.


## Project 4: Generative Modeling

In this project, we will be exploring different approaches for generative modeling. In the first part, we will implement a VAE to draw samples of handwritten digits. In the second part, we will learn a diffusion model to sample from a synthetic 2-D dataset.

**WHAT YOU'LL SUBMIT**: Your submission to Gradescope includes:
1. Your completed `submission.py` file uploaded to ***Coding Assignment 4***.
2. A `.pdf` file with responses to questions in the notebook and your **generated images** uploaded to ***Coding Assignment 4 Responses***.


<h3>Evaluation</h3>

<p><strong>This project must be successfully completed and submitted in order to receive credit for this course. Your score on this project will be included in your final grade calculation.</strong><p>
    
<p>You are expected to write code in the <em> #TODO </em> and <em> ####### </em> sections within the cells of this notebook. Only cells in which you write code will be graded. Be sure not to change the names of any provided functions, classes, or variables within the existing code cells, as this will interfere with grading. Also, remember to execute all code cells sequentially, not just those you’ve edited, to ensure your code runs properly.</p>
    
<p>You can resubmit your work as many times as necessary before the submission deadline. If you experience difficulty or have questions about this exercise, use the Q&A discussion board to engage with your peers or seek assistance from the instructor.<p>

Please install all of the necessary packages shown in the code box below


In [ ]:
!pip install datasets
!pip install einops
!pip install transformers

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from torchvision.utils import save_image, make_grid
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import random
import os
import math
from collections import namedtuple
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from functools import partial
import transformers
import matplotlib.animation
from IPython.display import HTML
from IPython.display import clear_output
import time
from google.colab import drive

import sys

random.seed(0)
torch.manual_seed(0)

In [ ]:
# TODO: Mount your Google Drive; this allows the runtime environment to access your drive.
drive.mount('/content/gdrive')

# NOTE: Make sure your path does NOT include a '/' at the end!
base_dir = "/content/gdrive/MyDrive/"
sys.path.append(base_dir)
## END TODO

In [ ]:
# This makes sure the submission module is reloaded whenever you make edits.
%load_ext autoreload
%aimport submission
%autoreload 1
import submission

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(F"Device set to {device}")

# Part 1: Variational Autoencoder (VAEs) (50 pts)

In this part, you will be training a varianational autoencoder on the MNIST dataset, which consists of a large collection of 28x28 pixel grayscale images of handwritten digits (0-9). Upon successful training, the VAE will be capable of generating new digit images that look like those in the dataset.


In [ ]:
# set up the dataset and dataloaders

# cuatom dataset
class MNISTDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]['image']
        image = self.transform(image)
        label = self.data[idx]['label']

        return image, label

transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset = load_dataset('mnist')
train_dataset = MNISTDataset(dataset['train'], transform)
test_dataset = MNISTDataset(dataset['test'], transform)

# dataloaders
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)

### Part 1.1: VAE Implementation (20 pts)

You will be implementing a VAE as depicted in the diagram below.

<center><img src="https://drive.google.com/uc?export=view&id=1JLvO2yVDUOsb-MHQdXDuPrUx0L9huQ7f" width="650" height="400"/></center>

Here are more details:

1. The input to the VAE is 28 x 28 dimensional MNIST images.
2. The `__init__()` method defines the following layers:
3. Two fully connected encoder layers- the first one produces `h_dim1=512` features and second one produces `h_dim2=256` features (you should be able to figure out the number of input features).
4. A fully connected layer that maps the `h_dim2=256` features to `z_dim=32` features that representa mean.
5. Another fully connected layer that map the `h_dim2=256` features to `z_dim=32` features that representa the log variance.
6. Two fully connected decoder layers- the first one produces `h_dim1=256` features and second one produces `h_dim2=512` features (you should be able to figure out the number of input features).
7. An output fully connected layer that produces `x_dim=784` features.
8. `encoder()` defines the encoder forward pass and returns the mean ${\boldsymbol{\mathbf{\mu}}}$ and log variance $\log {\boldsymbol{\mathbf{\sigma}}}^2$ for ever sample in the batch.
9. `decoder()` defines the decoder forward pass and should return the output which has `x_dim=784` features. **Note: Add a sigmoid activation before you return the output to ensure that the values produced are between 0 and 1.**
10. `sample()` returns the sampled latent vectors. Dimension $i$ of the vector should be sampled from $\mathcal{N}(\mu_i, \sigma_i)$. You we will need to use the re-parametrization trick as discussed in class. More details are provided below in `sample()`.
11. `forward()` calls `encoder()`, `sample()`, and `decoder()` to return the flattended reconstruction, the mean and log_variance vectors.


**Implement VAE in `submission.py`** then run the following:


In [ ]:
from submission import VAE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=32)
if torch.cuda.is_available():
    vae.to(device)

### Part 1.2: VAE Loss Implementation (10 pts)
The VAE loss is a sum of the following two terms:

1. The reconstruction loss: pixel-wise binary cross entropy beween the flattened reconstructed image `recon_x` and the flattened original image `x`. So, for each pixel, the prediction is obtainined from the reconstructed image and the label is the corresponding pixel in the original image. This loss across all the pixels is aggregated by summing the loss from each pixel.
2. KL Divergence Loss: Using a standard gaussian prior $p(z)$ and the encoder's distribution $q_{\theta} (z|x_i)$, you need to compute $KL(q_{\theta} (z|x_i)||p(z))$. Note that $p(z)$ and $q_{\theta} (z|x_i)$ are both multi-variate gaussian distributions. Suppose $q_{\theta} (z|x_i)$ and $p(z)$ have means ${\boldsymbol {\mu }}_{0}$ and ${\boldsymbol {\mu }}_{1}$, and covariace ${\boldsymbol {\Sigma }}_{0}$ and ${\boldsymbol {\Sigma }}_{1}$ respectively, their KL divergence can be computed as follows (https://en.m.wikipedia.org/wiki/Multivariate_normal_distribution)):
   ${\displaystyle D_{\text{KL}}({\mathcal {N}}_{0}\parallel {\mathcal {N}}_{1})={1 \over 2}\left\{\operatorname {tr} \left({\boldsymbol {\Sigma }}_{1}^{-1}{\boldsymbol {\Sigma }}_{0}\right)+\left({\boldsymbol {\mu }}_{1}-{\boldsymbol {\mu }}_{0}\right)^{\rm {T}}{\boldsymbol {\Sigma }}_{1}^{-1}({\boldsymbol {\mu }}_{1}-{\boldsymbol {\mu }}_{0})-k+\ln {|{\boldsymbol {\Sigma }}_{1}| \over |{\boldsymbol {\Sigma }}_{0}|}\right\}}$

When computing the KL divergence loss, the constant $k$ can be ignored because it is just a constant. **Simplify the equation above and write the KL Divergence Loss as a function of the provided `mu` and `log_var`.**

Hint 1: $p(z)$ is a standard gaussian.

Hint 2: The covariance matrix ${\boldsymbol {\Sigma }}_{1}$ is a diagonal matrix whose entries can be found by using `log_var`.

Hint 3: You don't need to do any 2d matrix multiplications after simplifying.

**Note: Make sure there are no loops and no 2d matrix multiplications in your code for computing the loss. For both loss terms, compute the loss over the entire batch by summing the loss from each sample.**


**Implement `loss_function` in `submission.py`** then run the following:


In [ ]:
optimizer = optim.Adam(vae.parameters())
from submission import loss_function

In [ ]:
best = 1000000
# train and test code
def train(model, epochs):
    for epoch in range(1, epochs):
      model.train()
      train_loss = 0
      for batch_idx, (data, _) in enumerate(train_loader):
          data = data.to(device)
          optimizer.zero_grad()

          recon_batch, mu, log_var = model(data)
          loss = loss_function(recon_batch, data.view(-1, 784), mu, log_var)

          loss.backward()
          train_loss += loss.item()
          optimizer.step()

          if batch_idx % 100 == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item() / len(data)))
        
      print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))
      test(model)

    return model

def test(model):
    model.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            recon, mu, log_var = model(data)

            # sum up batch loss
            test_loss += loss_function(recon, data.view(-1, 784), mu, log_var).item()

    test_loss /= len(test_loader.dataset)
    global best
    if test_loss < best:
        best = test_loss
        torch.save(model.state_dict(), os.path.join(base_dir, f'best.pth'))
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
# Train the model for 10 epochs
vae = train(vae, 11)

### Part 1.3: Sample images from VAE (20 pts)
Sample images from the VAE. To sample an image, you can sample vectors from a standard gaussian and pass it through the decoder. The `torch.randn()` function will be helpful for this part.


**Implement `sample_images` in `submission.py`** then run the following:


In [ ]:
from submission import sample_images

samples = sample_images(vae, num_samples=64)

In [ ]:
# visualize the generated samples
def plot_images(images, rows=8, cols=8, figsize=(10, 10)):
    images = images.view(64, 28, 28)
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].cpu().numpy())
        ax.axis('off')
    plt.tight_layout()
    plt.show()

plot_images(samples)

### Q: Do you see all 10 digits? Do you notice that some digits are better quality than others? How would you try to improve the quality of the digits?

> Indented block


**Notes:** complete `get_embeddings()` function below:


In [ ]:
# get embeddings for the first 100000 samples
def get_embeddings():
    vae.eval()
    test_loss= 0
    embeddings = []
    embedding_labels = []
    with torch.no_grad():
        for i, (data, labels) in enumerate(test_loader):
            data = data.to(device)
            ## TODO: obtain the 32-dimensional latent embedding vector for each
            #        sample in the batch and append the batch_size x 32
            #        dimensional matrix to the embeddings list
            mu, log_var = vae.encoder(data)
            embeddings.append(vae.sample(mu, log_var))

            #################
            embedding_labels.append(labels)
            if i==99:
              break
    embeddings = torch.concatenate(embeddings)
    embedding_labels = torch.concatenate(embedding_labels)
    return embeddings, embedding_labels

In [ ]:
# Reduce the dimensionality of the obtained embeddigns to 2 and plot it on a
# T-SNE Plot
embeddings, embedding_labels = get_embeddings()
embeddings = embeddings.cpu()
embedding_labels = embedding_labels.cpu()

print("Running TSNE to reduce dimensionality")

tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings)

plt.figure(figsize=(10, 8))
for label in range(10):
    plt.scatter(embeddings_2d[embedding_labels == label, 0],
                embeddings_2d[embedding_labels == label, 1],
                label=str(label), alpha=0.5)
plt.title('t-SNE plot of MNIST embeddings')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend()
plt.grid(True)
plt.show()

### Q: Do you notice that clusters in the center are smaller than the peripheral clusters on average? If so, why do you think is the case?

> Indented block


# Part 2: Diffusion Models (50 pts)

In this section, you will develop a diffusion model to generate samples from a 2D synthetic dataset. Specifically, we will be using a dataset which consists of three rings and is visualized below.


<center><img src="https://drive.google.com/uc?export=view&id=1pU2sne-Fvw82dYp2IwGw4dsFlTKzduzx"
width="400"
height="400"/>
</center>


We provide some helper functions below that will be used throughout the assignment. We also define a namedtuple, `ModelPrediction` that will store the prediction of our diffusion network at any given timestep.


In [ ]:
def right_pad_dims_to(x, t):
    padding_dims = x.ndim - t.ndim
    if padding_dims <= 0:
        return t
    return t.view(*t.shape, *((1,) * padding_dims))

def exists(val):
    return val is not None

ModelPrediction =  namedtuple('ModelPrediction', ['pred_noise', 'pred_x_start'])

## Diffusion Background

Diffusion models are latent variable models with latents $\mathbf{z}  = \{\mathbf{z}_t | t\in [0,1] \}$ given by a forward diffusion process $q(\mathbf{z}|\mathbf{x})$, which defines a gradual transition from the data distribution, $\mathbf{x} \sim p(\mathbf{x})$, to a Gaussian distribution. The Markovian forward process iteratively adds Gaussian noise to the data over time and satisfies
\begin{align*}
q(\mathbf{z}*t|\mathbf{z}\_s)=\mathcal{N}(\mathbf{z}\_t; \alpha*{t|s}\mathbf{z}*s, (1-\alpha*{t|s}^2)\mathbf{I}),\\
q(\mathbf{z}\_t|\mathbf{x}) = \mathcal{N}(\mathbf{z}\_t; \alpha_t\mathbf{x}, (1-\alpha_t^2)\mathbf{I})
\end{align*}
where $\alpha_{t|s} = \alpha_t/\alpha_s$
and $0 \leq s < t \leq 1$. Using the reparameterization trick, we can also write
\begin{align*}
\mathbf{z}\_t = \alpha_t\mathbf{x} + \sqrt{(1-\alpha_t^2)}\epsilon, \quad \epsilon \sim \mathcal{N}(\mathbf{0}, \mathbf{I})
\end{align*}

The noise schedule, determined by $\alpha_t\in [0,1]$, monotonically decreases the signal-to-noise ratio (SNR), $\lambda_t =\frac{\alpha_t^2}{1-\alpha_t^2}$ as a function of the time, $t$, such that the initial latent is close to the original data, $\mathbf{z}_0 \approx \mathbf{x}$, and the final latent becomes approximately Gaussian, $q(\mathbf{z}_1) \approx \mathcal{N}(\mathbf{0}, \mathbf{I})$. The forward process therefore defines a transition from the data distribution to a Gaussian distribution.

Diffusion models define a generative process to invert the forward process. This specifies a transition from Gaussian noise, which can be sampled analytically, to the unknown data distribution. Inverting this process can be reduced to learning a denoising network $$\hat{\mathbf{x}}_\theta(\mathbf{z}_t, t) \approx \mathbf{x}$$ that reconstructs the clean data given some noisy latent and the time.

In practice, people have found that parameterizing the denoising network as a noise prediction network $$\hat{\mathbf{\epsilon}}_\theta(\mathbf{z}_t, t) \approx \mathbf{\epsilon}$$ improves performance. Instead of predicting the clean data, the noise prediction network predicts the added noise. We therefore train the network with the following regression objective
\begin{align*}
\mathcal{L}(\theta) = \mathbb{E}*{t,\mathbf{x}, \epsilon} [ \lVert\hat{\mathbf{\epsilon}}_{\theta}(\mathbf{z}\_t, t) - \mathbf{\epsilon} \rVert_2^2
\end{align_}

This loss function is the weighted variational lower bound of the log likelihood of the data under the forward diffusion process.


## Part 2.1: Diffusion Helper Functions (10 pts)

1. We will use the popular cosine noise schedule that sets $\alpha_t = \cos(.5 \pi t)$. Implement the `cosine_schedule()` function to map the timestep $t\in[0,1]$ to $\alpha_t^2$.

2. Predicting the noise added to the data also provides us with an estimate of the clean data $$\hat{\mathbf{x}}_\theta(\mathbf{z}_t, t)$$ which we will use for sampling. Given $$\mathbf{z}_t, \alpha_t^2, \hat{\mathbf{\epsilon}}_\theta(\mathbf{z}_t, t)$$ there is a closed-form solution for the estimated clean data. This closed-form solution can be derived directly from an equation provided in the background section. Implement `predict_start_from_noise()` to compute and return this estimate. Note that you should clamp any denominators to a minimum value of `clamp_min` to avoid dividing by numbers very close to zero.


**Implement `cosine_schedule(), predict_start_from_noise()` in `submission.py`** then run the following to import them here:


In [ ]:
from submission import cosine_schedule, predict_start_from_noise

## Part 2.2: Diffusion Training and Sampling (40 pts)

We define a `GaussianDiffusion()` class in `submission.py` that handles training and sampling.

### Part 2.2.1: Training (20 pts)

We train the network with the following regression objective
\begin{align*}
\mathcal{L}(\theta) = \mathbb{E}*{t,\mathbf{x}, \epsilon} [ \lVert\hat{\mathbf{\epsilon}}_{\theta}(\mathbf{z}\_t, t) - \mathbf{\epsilon} \rVert_2^2.
\end{align_}
For this part, you will finish implementing the `forward()` function. You need to:

1. Compute the noisy latent $z_t$.
2. Compute the diffusion model predictions with `self.diffusion_model_predictions()`. The function accepts the noisy latents and the timesteps.
3. Implement the regression loss from the background section. The output of the network should be the noise $\epsilon \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$ use the create the noisy latent $\mathbf{z}_t$.

**Note: Implement `forward()` in `submission.py`.**

### Part 2.2.2: Sampling (20 pts)

In this part, you will implement the DDIM sampler, which is used for the reverse process of the diffusion model. The DDIM sampler enables sampling from a noisy latent representation $z_t$ at a higher noise level (time step $t$) to a less noisy latent representation $z_s$ at a lower noise level (time step $s$, where $s < t$). This gradual denoising of latent representations is achieved through the following equation:

$$ z*s = \alpha_s \hat{\mathbf{x}}*\theta(\mathbf{z}_t, t) + \sqrt{1 - \alpha_s^2} \hat{\mathbf{\epsilon}}_\theta(\mathbf{z}\_t, t).$$

Your task is to complete the `ddim_sample()` function in the `GaussianDiffusion` class to implement the DDIM sampler for the reverse process.

**Note: Implement `ddim_sample()` in `submission.py`.**


### Q: What are the sources of stochasticity in the DDIM sampler? How do different samples from the same initial draw of noise relate to each other, if at all? Write 2-3 sentences below.


In [ ]:
from submission import GaussianDiffusion

## Part 2.3: Diffusion Model Architecture (5 pts)

For this part, we provide the diffusion architecture for you. This diffusion architecture accepts the noisy latent and the timestep and predicts the added noise.


### Q: Briefly describe the diffusion architecture being used for this problem? How is the timestep information being incorporated? Write 3-4 sentences below.


In [ ]:
def zero_init_(m):
    nn.init.zeros_(m.weight)
    if exists(m.bias):
        nn.init.zeros_(m.bias)

class SwiGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim = -1)
        return F.silu(gate) * x

class RMSNorm(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.scale = dim ** 0.5
        self.gamma = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        out = F.normalize(x, dim = -1) * self.scale * self.gamma
        return out

class FeedForward(nn.Module):
    def __init__(
        self,
        dim,
        mult = 4,
        time_cond_dim = None,
        dropout = 0.,
    ):
        super().__init__()
        self.norm = RMSNorm(dim)
        inner_dim = dim * mult
        dim_out = dim

        self.time_cond = None
        self.dropout = nn.Dropout(dropout)

        self.net = nn.Sequential(
            nn.Linear(dim, inner_dim*2),
            SwiGLU(),
            nn.Dropout(dropout),
            nn.Linear(inner_dim, dim_out)
        )

        if exists(time_cond_dim):
            self.time_cond = nn.Sequential(
                nn.SiLU(),
                nn.Linear(time_cond_dim, dim * 3),
                Rearrange('b d -> b d')
            )

            zero_init_(self.time_cond[-2])
        else:
            zero_init_(self.net[-1])


    def forward(self, x, time = None):
        x = self.norm(x)
        if exists(self.time_cond):
            assert exists(time)
            scale, shift, gate = self.time_cond(time).chunk(3, dim = 1)
            x = (x * (scale + 1)) + shift

        x = self.net(x)

        if exists(self.time_cond):
            x = x*gate

        return x

class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = x[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb

class ScoreNet(nn.Module):
    def __init__(
        self,
        dim,
        n_layers = 3,
        *,
        time_cond_dim = None,
        dropout = 0.
    ):
        super().__init__()
        if not exists(time_cond_dim):
            time_cond_dim = dim

        self.init_proj = nn.Linear(2, dim)
        sinusoidal_pos_emb = SinusoidalPosEmb(dim)
        self.time_mlp = nn.Sequential(
            sinusoidal_pos_emb,
            nn.Linear(time_cond_dim, time_cond_dim),
            nn.SiLU(),
            nn.Linear(time_cond_dim, time_cond_dim)
        )
        self.nets = nn.ModuleList([FeedForward(dim, time_cond_dim = time_cond_dim, dropout = dropout) for _ in range(n_layers)])
        self.norm = RMSNorm(dim)
        self.output_proj = nn.Linear(dim, 2)

    def forward(self, x, alpha2 = None):

        time_cond = self.time_mlp(alpha2*1000)

        x = self.init_proj(x)

        for net in self.nets:
            x = x + net(x, time = time_cond)

        return self.output_proj(self.norm(x))

This block of code defines the synthetic 2d dataset that we will be using to train our diffusion model.


In [ ]:
def normalize(points):
    """
    Normalize so that the points have zero mean and unit variance.
    :param points: (N, 2)
    """
    mean = points.mean()
    std = points.std()
    return (points - mean) / std

class BaseSampler(object):
    def __init__(self, radii: np.array, centers: np.array, width: float):
        """
        Base sampler for rings and squares.
        :param radii: radius of the rings or squares
        :param centers
        :param width: the width of each ring / square
        """
        self.num_objects = radii.shape[0]  # Number of rings or squares
        self.radii = np.array(radii, np.double)
        self.centers = np.array(centers, np.double)
        self.width = width

class RingSampler(BaseSampler):
    """
    Data sampler class to generate synthetic 2D distribution data.
    The implementation considers only circle-like distributions.
    """

    def sample(self, N):
        """
        Samples from the 2D distribution.
        Returns a sample of the shape (N, 2).
        :param N: number of data points
        :return:
        """
        # Assigns points to one of the K rings
        K = self.num_objects
        assert N % K == 0
        indices = np.arange(0, K)
        indices = np.tile(indices, N // K + 1)[:N]
        indices = np.sort(indices)  # (0,0,0,0,1,1,1,1...
        assert indices.shape[0] == N
        centers = self.centers[indices]

        radii_eps = (np.random.rand(N) - .5) * self.width
        radii = self.radii[indices] + radii_eps

        # Randomly assigns points on the ring
        theta = np.random.rand(K, N // K) * 2 * np.pi
        theta = np.sort(theta, 1).reshape(-1)[::-1]
        x, y = np.rollaxis(centers, 1)
        px = radii * np.cos(theta) + x
        py = radii * np.sin(theta) + y

        points = np.stack([px, py], axis=1)
        points = normalize(points)
        return points, indices

class OlympicRingSampler(RingSampler):
    def __init__(self, radii: np.array = np.ones(5), width: float = 0.5):
        # The ratio between radii and centers is fixed
        num_objects = radii.shape[0]
        centers = np.array([(-140, 0), (0, 0), (140, 0), (-55, -50), (55, -50)], np.float32) / float(50)
        centers = centers[:num_objects]
        super(OlympicRingSampler, self).__init__(radii, centers, width)


def scatter(points, enable_color_interpolation=True):
    """Draws a scatter, fine plots of the given points."""
    xlim, ylim = 3, 3
    N = points.shape[0]
    px, py = np.rollaxis(points, 1)
    plt.figure(figsize=(6, 6))
    plt.xlim(-xlim, xlim)
    plt.ylim(-ylim, ylim)
    ax = plt.gca()
    ax.set_aspect('equal')
    if enable_color_interpolation:
        colors = np.arange(0, N)
    else:
        colors = np.array([[0.525776, 0.833491, 0.288127]])
    plt.scatter(px, py, s=1.0, marker='o', c=colors, cmap='rainbow', linewidths=0, alpha=1.0)


class Synthetic2DDataset(Dataset):
    def __init__(self, n_samples,):
        super().__init__()
        self.n_samples = n_samples

        radii_uniform = np.array([1, 1, 1])
        points, indices = OlympicRingSampler(radii_uniform).sample(n_samples)
        self.points = normalize(points).astype(np.float32)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, index):
        return np.array(self.points[index])


This cell visualizes the synthetic dataset that we will be using to train the model. The colors are purely for visualization purposes and are not meaningful.


In [ ]:
N = 150000
shape = 'parallel_rings'
print(f"Plotting {shape} images.")
dataset = Synthetic2DDataset(N)
points = dataset.points
print(points.mean(), points.std())
scatter(points)

# save image
plt.savefig(f"part2_1.png", dpi=300, bbox_inches='tight')

## Part 2.4 Diffusion Model Training and Evaluation (5 pts)

We provide the code to train the diffusion model and draw samples from it. We
then visualize the samples generated by the network.


In [ ]:
# Training Loop
def train_loop(model, loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in tqdm(loader):
        batch = batch.to(device)
        optimizer.zero_grad()
        loss = model(batch)
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
# Create training dataset

n_samples = 500001
dataset = Synthetic2DDataset(n_samples)
loader = DataLoader(dataset, batch_size=256, shuffle=True)


In [ ]:
model = GaussianDiffusion(ScoreNet(256, n_layers=6))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
NUM_EPOCHS = 25
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=NUM_EPOCHS * len(loader))

In [ ]:
model.train()
for idx in range(NUM_EPOCHS):
    loss = train_loop(model, loader, optimizer, scheduler, device)
    print(f'Epoch {idx}: loss {loss}')

In [ ]:
model.eval()
sampled_points, intermediate_preds = model.sample(50000, 1024)

In [ ]:
sampled_points.shape

In [ ]:
intermediate_preds.shape

In [ ]:
def sort_by_angles(arr):
    # Convert cartesian coordinates to polar coordinates
    r = np.sqrt(arr[:, 0]**2 + arr[:, 1]**2)
    theta = np.arctan2(arr[:, 1], arr[:, 0])

    # Combine angles and original data points
    polar_arr = np.column_stack((theta, arr))

    # Sort the array based on angles
    sorted_indices = polar_arr[:, 0].argsort()

    return sorted_indices


In [ ]:
sort_idx = sort_by_angles(intermediate_preds[:,0,:])

## Diffusion Sample Visualization

We visualize the samples drawn from the diffusion model.


In [ ]:
scatter(sampled_points[sort_idx].to('cpu').numpy())
plt.savefig(f"part2_2.png", dpi=300, bbox_inches='tight')

## Diffusion Process Visualization

We visualize the evolution of the samples throughout the diffusion process. Points of the same color correspond to the same latent across time throughout the diffusion process.


In [ ]:
for idx in range(intermediate_preds.shape[1]):
    scatter(intermediate_preds[:,idx,:][sort_idx])
    if idx == intermediate_preds.shape[1]-1:
        plt.savefig(f"part2_3.png", dpi=300, bbox_inches='tight')


### Q: What do you observe about the diffusion sampling process? Does there appear to be any relationship between the initial Gaussian latent variable $\mathbf{z}_1$ and the final sample from the data distribution $\mathbf{z}_0$. Write 3-4 sentences.
